# **Metra Stations and Ridership**
*By: Emily Thompson*


This project looks at ridership metrics across Metra stations, as well as the demographic context of populations near those stations. Ridership survey data, station locations, and US Census tract populations (for tracts within 1 mile of Metra stations, in Illinois, Indiana, and Wisconsin) are integrated to produce per capita ridership metrics and an interactive map for visualization. Currently the script is set up so that the map shows baseline per capita ridership at each station (boards and alights), and colors and sizes them thusly, but it could later be modified to show additional demographic contexts.

The workflow includes the following steps:

**Load and clean datasets**
- Metra ridership by station and year (1979–2018), boards and alights. [Source](https://rtams.org/sites/default/files/documents/2024-07/Metra_Ridership_by_Station_Boarding_Alighting%20Survey_1979_2018.csv)
- Metra station shapefile with station geometries. [Source](https://data.cityofchicago.org/download/nqm8-q2ym/application%2Fzip)
- Census tract shapefiles for Illinois, Indiana, and Wisconsin. [Source](https://pub-a835f667d17f4b6691fafec7e9ede33d.r2.dev/tiger/GENZ2020/shp/cb_2020_17_tract_500k.zip)
- 2020 Census P1 population data (aquired via API call). [Source](https://api.census.gov/data/2020/dec/pl?get=group(P1)&ucgid=pseudo(0400000US17$1400000,0400000US18$1400000,0400000US55$1400000))

**Join ridership and station data**
- Normalize station names to align the ridership and shapefile records - some manual edits required before getting in here.
- Merge ridership counts with station geometries.
- Handle missing or mismatched stations (e.g., Romeoville was missing from the .shp).

**Assign population to stations**
- Reproject data into a metric CRS.
- Generate 1-mile buffers around each station.
- Perform a spatial join between buffers and census tracts.
- Aggregate population totals for each station’s buffer zone.

**Calculate per-capita ridership metrics**
- Select a year of interest (2018 in this version).
- Aggregate total boardings and alightings per station.
- Divide by population within 1 mile of each station to compute boardings per capita and alightings per capita.

**Interactive visualization**
- Convert geometries to WGS84 (EPSG:4326) for mapping.
- Render stations as circle markers on a Folium basemap.
- Scale circle size by ridership variable: boards, alights, boards per capita, and alights per capita.
- Have different map tabs for the different ridership data variables.
- Color-code markers with a continuous colormap (with legend that also indicates year)
- Provide popups when clicking on the circles showing station name, boardings, alightings, assigned population, and per-capita values.

**Outcome**

The result is an interactive map that highlights:
- Which stations have the highest and lowest ridership relative to surrounding population density.
- Comparative demand across all Metra stations.
- Potential gaps where service might be expanded.

In [1]:
# install  packages
!pip install geopandas folium mapclassify requests shapely pyproj python-Levenshtein


   ---------------------------------------- 0.0/882.2 kB ? eta -:--:--
   --------------------------------------- 882.2/882.2 kB 28.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 60.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 76.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 60.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------  19.1/19.2 MB 98.2 MB/s eta 0:00:01
   ---------------------------------------- 19.2/19.2 MB 85.6 MB/s eta 0:00:00

   ----------------------------------------  0/10 [shapely]
   ----------------------------------------  0/10 [shapely]
   ---- -----------------------------------  1/10 [rapidfuzz]


In [2]:
# imports
import io
import zipfile
import requests
import tempfile
from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import numpy as np
import difflib
import folium
from folium.plugins import MarkerCluster
import mapclassify
import math
import json
from branca.colormap import linear

# data sources
METRA_RIDERSHIP_CSV = "https://raw.githubusercontent.com/emilythompson-gis/Metra-Data/refs/heads/main/Data/metra_ridership.csv"
METRA_STATIONS_ZIP = "https://github.com/emilythompson-gis/Metra-Data/raw/refs/heads/main/Data/metra_stations/MetraStations.shp"
IL_CENSUS_TRACTS_ZIP = "https://github.com/emilythompson-gis/Metra-Data/raw/refs/heads/main/Data/census_tracts.zip"
IN_CENSUS_TRACTS_ZIP = "https://github.com/emilythompson-gis/Metra-Data/raw/refs/heads/main/Data/indiana_tracts.zip"
WI_CENSUS_TRACTS_ZIP = "https://github.com/emilythompson-gis/Metra-Data/raw/refs/heads/main/Data/wisco_tracts.zip"
CENSUS_P1_API = "https://github.com/emilythompson-gis/Metra-Data/raw/refs/heads/main/Data/demographic_data.json"

# 1 mile buffer in meters
BUFFER_METERS = 1609.344


In [3]:
# load ridership CSV
ridership = pd.read_csv(METRA_RIDERSHIP_CSV)
ridership.columns = ["STATION", "YEAR", "BOARDS", "ALIGHTS"]
ridership['STATION'] = ridership['STATION'].astype(str).str.strip()
ridership['YEAR'] = pd.to_numeric(ridership['YEAR'], errors='coerce').astype('Int64')

# preview
print("ridership rows:", len(ridership))
ridership.head()


ridership rows: 3453


,STATION,YEAR,BOARDS,ALIGHTS
0,103rd Street (Beverly Hills),2018,734,731
1,103rd Street (Beverly Hills),2016,759,754
2,103rd Street (Beverly Hills),2014,767,806
3,103rd Street (Beverly Hills),2006,1150,1210
4,103rd Street (Beverly Hills),2002,977,986


In [4]:
# get metra_stations shapefile (zipped)
stations_gdf = gpd.read_file(METRA_STATIONS_ZIP)
stations_gdf.columns = stations_gdf.columns.str.strip()
stations_gdf['LONGNAME'] = stations_gdf['LONGNAME'].astype(str).str.strip()

print("stations:", len(stations_gdf))
stations_gdf.head()


stations: 248


,OBJECTID,STATION_ID,ASSET_ID,NAME,LONGNAME,LINES,BRANCH_ID,STATUS,MILEPOST,FAREZONE,...,TICKET_AVA,ADDRESS,MUNICIPALI,TELEPHONE,WEBLINK,LABELANGLE,EDIT_INIT,EDIT_DATE,YEAR_OPEN,geometry
0,1,1091.0,51201091.0,Stony Island,Stony Island,Electric,1.0,1.0,9.1,B,...,None,71st St. (at Stony Island Ave.),Chicago,None,None,30.0,ks,1995-08-02,None,MULTIPOINT ((1187895.88 1858208.72))
1,2,1097.0,51201097.0,Bryn Mawr,Bryn Mawr,Electric,1.0,1.0,9.7,B,...,None,71st St. (at Jeffrey Blvd.),Chicago,None,None,30.0,ks,1995-08-02,None,MULTIPOINT ((1190583.72 1858270.34))
2,3,1103.0,51201103.0,South Shore,South Shore,Electric,1.0,1.0,10.3,B,...,None,71st St. (near Yates and South Shore Dr.),Chicago,None,None,0.0,ks,1995-08-02,None,MULTIPOINT ((1193664.51 1857979.74))
3,4,1109.0,51201109.0,Windsor Park,Windsor Park,Electric,1.0,1.0,10.9,B,...,None,75th St. (at Exchange Ave.),Chicago,None,None,0.0,ks,1995-08-02,None,MULTIPOINT ((1195346.74 1855540.54))
4,5,1115.0,51201115.0,Cheltenham,Cheltenham (79th Street),Electric,1.0,1.0,11.5,B,...,None,79th St. (at Exchange Ave.),Chicago,None,None,0.0,ks,1995-08-02,None,MULTIPOINT ((1197216.28 1853302.53))


In [5]:
# join ridership -> stations using normalized names
# in the original data, some of the names between metra_ridership and metra_stations did not match (different format or missing parentheticals suchs as "Belmont Ave. (Franklin Park)" was just "Belmont Ave")
# Also the Romeoville stop was missing from the metra_stations shapefile, so added that point using QGIS
# try exact merge on normalized names
stations_gdf['join_name'] = stations_gdf['LONGNAME'].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()
ridership['join_name'] = ridership['STATION'].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

# exact merge
merged_exact = ridership.merge(stations_gdf.drop(columns=['STATION'] , errors='ignore'), left_on='join_name', right_on='join_name', how='outer', suffixes=('_ridership', '_station'))

# check how many ridership rows have no geometry (no station found)
missing = merged_exact[merged_exact.geometry.isna()]
print(f"Exact merges missing: {len(missing)} of {len(ridership)} rows")

# Convert merged_all to GeoDataFrame (some rows may be duplicates if ridership contains many years)
gdf_stations_ridership = gpd.GeoDataFrame(merged_exact, geometry='geometry', crs=stations_gdf.crs)

print("Final joined rows:", len(gdf_stations_ridership))
gdf_stations_ridership.head()

# check if 2018 is missing - some older / historical stations are missing, this can be fixed later...
assert not (2018 in missing.YEAR.unique())

Exact merges missing: 106 of 3453 rows
Final joined rows: 3454


In [6]:
# get census tracts shapefile (zipped)
tracts_gdf = pd.concat([
    gpd.read_file(IL_CENSUS_TRACTS_ZIP),
    gpd.read_file(IN_CENSUS_TRACTS_ZIP),
    gpd.read_file(WI_CENSUS_TRACTS_ZIP)
])
tracts_gdf.columns = tracts_gdf.columns.str.strip()
print("tracts count:", len(tracts_gdf))


tracts count: 6501


In [7]:
# pull Census P1 data from API and join to tracts
r = requests.get(CENSUS_P1_API)
r.raise_for_status()
p1 = r.json()  # first row is columns, next rows are data
cols = p1[0]
rows = p1[1:]
p1_df = pd.DataFrame(rows, columns=cols)

# keep only GEO_ID and P1_001N (total population)
p1_df = p1_df[['GEO_ID', 'P1_001N']].copy()
p1_df['P1_001N'] = pd.to_numeric(p1_df['P1_001N'], errors='coerce').fillna(0).astype(int)
p1_df['GEO_ID'] = p1_df['GEO_ID'].astype(str).str.replace('1400000US','')

tracts_gdf = tracts_gdf.merge(p1_df, left_on='GEOID', right_on='GEO_ID')

In [8]:
# Reproject both datasets to a metric CRS for buffering (EPSG:3857 used here for simplicity)
metric_crs = "EPSG:3857"  # web mercator

stations_proj = gdf_stations_ridership.to_crs(metric_crs)
tracts_proj = tracts_gdf.to_crs(metric_crs)

# only using stations joined and have geometry
unique_stations = stations_proj.query('geometry.notna()')[[
    'join_name', 'STATION_ID', 'geometry'
]].drop_duplicates()

# create 1-mile buffer around each station geometry (one buffer per station location)
# buffer and make a GeoDataFrame of buffers
buffers_gdf = unique_stations.copy()
buffers_gdf['geometry'] = buffers_gdf.geometry.buffer(BUFFER_METERS)

print("Buffers created for stations:", len(buffers_gdf))
buffers_gdf.head()


Buffers created for stations: 248


,join_name,STATION_ID,geometry
0,103rd street (beverly hills),7128.0,"POLYGON ((-9757648.498 5117073.152, -9757656.2..."
15,103rd street (rosemoor),5130.0,"POLYGON ((-9750758.032 5117185.101, -9750765.7..."
30,107th street,5135.0,"POLYGON ((-9750941.887 5116122.411, -9750949.6..."
45,107th street (beverly hills),7133.0,"POLYGON ((-9757743.095 5115984.962, -9757750.8..."
60,111th street,5140.0,"POLYGON ((-9751115.137 5115117.164, -9751122.8..."


In [9]:
# spatial jawns

stations_join = gpd.sjoin(buffers_gdf, tracts_proj, how = 'left')[[
    'join_name','P1_001N'
]]
stations_grouped = stations_join.groupby('join_name').sum().reset_index()

# make sure all stations have data / population
assert len(stations_grouped.query('P1_001N.isna()')) == 0
assert len(stations_grouped.query('not (P1_001N > 0)')) == 0

#rename some things
station_populations = stations_grouped.rename(columns={
    'P1_001N':'population_1mile'
})

In [10]:
# choose year and compute per-capita metrics
YEAR = 2018  # picking 2018 to summarize

# aggregate ridership by station (station LONGNAME) for the chosen YEAR
ridership_year = ridership[ridership['YEAR'] == YEAR].copy()
# previously merged ridership to stations into gdf_stations_ridership, but to be safe:
# aggregate by join_name or STATION, then merge to station_id via LONGNAME matching
agg_r = ridership_year.groupby('join_name', as_index=False).agg({'BOARDS':'sum','ALIGHTS':'sum'})

# merge with population
agg_r = agg_r.merge(station_populations, left_on='join_name', right_on='join_name', how='left')

# compute per-capita (avoid division by zero), fill boards and alights per capita with 0 when null
# we know no stations have 0 pop now so getting rid of this agg_r['population_1mile'] = agg_r['population_1mile'].fillna(0).astype(float)
agg_r['boards_per_capita'] = agg_r.apply(lambda r: r['BOARDS']/r['population_1mile'], axis=1)
agg_r['alights_per_capita'] = agg_r.apply(lambda r: r['ALIGHTS']/r['population_1mile'], axis=1)

# attach geometry for mapping
# giving geometry to stations with no 2018 data so be shown as grey on the map
# that way we see all the stations even if they have no 2018 data (like for some reason none of the Indiana stations have 2018 data)
agg_r_gdf = unique_stations.merge(agg_r, on='join_name', how='outer')

# transform back to EPSG:4326 for folium mapping
agg_r_gdf_4326 = agg_r_gdf.to_crs("EPSG:4326").dropna(subset="geometry")

# sample
agg_r_gdf_4326[['join_name','join_name','BOARDS','ALIGHTS','population_1mile','boards_per_capita','alights_per_capita']].head()


,join_name,join_name,BOARDS,ALIGHTS,population_1mile,boards_per_capita,alights_per_capita
0,103rd street (beverly hills),103rd street (beverly hills),734.0,731.0,32260.0,0.022753,0.022660
1,103rd street (rosemoor),103rd street (rosemoor),36.0,34.0,18780.0,0.001917,0.001810
2,107th street,107th street,27.0,23.0,24067.0,0.001122,0.000956
3,107th street (beverly hills),107th street (beverly hills),395.0,368.0,35167.0,0.011232,0.010464
4,111th street,111th street,31.0,49.0,27985.0,0.001108,0.001751


In [11]:
# create folium map and add stations as circles scaled by boards_per_capita
#first ensure all geometries are single Points (convert MultiPoint -> centroid)
agg_r_gdf_4326['geometry'] = agg_r_gdf_4326['geometry'].apply(
    lambda g: g.centroid if g.geom_type == 'MultiPoint' else g
)

# baseline map center: mean of station coordinates
mean_lat = agg_r_gdf_4326.geometry.y.mean()
mean_lon = agg_r_gdf_4326.geometry.x.mean()

ridership_variable_dictionary = {
    'Boards': 'BOARDS',
    'Alights': 'ALIGHTS',
    'Boards per Capita': 'boards_per_capita',
    'Alights per Capita': 'alights_per_capita'
}

# define a ridership variable so that the map can have different data tabs
def display_map(ridership_variable):
    m = folium.Map(
        location=[mean_lat, mean_lon],
        zoom_start=10,
        tiles="CartoDB Positron")
    ridership_column = ridership_variable_dictionary[ridership_variable]

    # create color scale for boards_per_capita
    # handle NaNs and zeros
    valid_vals = agg_r_gdf_4326[ridership_column].dropna()
    if len(valid_vals):
        vmax = valid_vals.max()
        vmin = valid_vals.min()
    else:
        vmax, vmin = 0, 0

    colormap = linear.YlOrRd_09.scale(vmin if not math.isnan(vmin) else 0, vmax if not math.isnan(vmax) else 1)
    colormap.caption = f"{ridership_variable} ({YEAR})"
    colormap.add_to(m)

    # cluster markers to keep map tidy
    marker_cluster = MarkerCluster()
    m.add_child(marker_cluster)

    # function to convert per-capita to radius in meters for CircleMarker
    def radius_from_value(v, scale=5000):
        # use sqrt scaling to avoid huge circles
        if v is None or (isinstance(v, float) and math.isnan(v)):
            return 3
        return max(3, math.sqrt(v) * scale)

    if ridership_variable in ['Boards', 'Alights']:
        scale=(3/10)
    else:
        scale=50

    for idx, row in agg_r_gdf_4326.iterrows():
        lat = row.geometry.y
        lon = row.geometry.x
        bp = row.get('boards_per_capita', None)
        ap = row.get('alights_per_capita', None)
        boards = int(row.get('BOARDS', 0)) if not pd.isna(row.get('BOARDS')) else 0
        alights = int(row.get('ALIGHTS', 0)) if not pd.isna(row.get('ALIGHTS')) else 0
        pop = int(row.get('population_1mile', 0)) if not pd.isna(row.get('population_1mile')) else 0
        name = row.get('join_name', '')
        display_variable = row.get(ridership_column)
        radius = radius_from_value(display_variable, scale=scale)
        color = colormap(display_variable) if (pd.notna(display_variable) and vmin != vmax) else "#808080"
        popup_html = f"""
        <b>{name}</b><br/>
        Year: {YEAR}<br/>
        Boards: {boards:,}<br/>
        Alights: {alights:,}<br/>
        Population within 1 mile: {pop:,}<br/>
        Boards per capita: {bp}<br/>
        Alights per capita: {ap}
        """
        folium.CircleMarker(
            location=(lat, lon),
            radius=radius,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=400)
        ).add_to(m)

    # display map
    return m


In [12]:
#and now we map it

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

title = widgets.HTML("<h1>🚂 Ridership Map Explorer</h1>")

display(title)
interact(
    display_map,
    ridership_variable=widgets.ToggleButtons(
        options=['Boards', 'Alights', 'Boards per Capita', 'Alights per Capita'],
        description='Select a ridership metric to view:',
        button_style='',  # can use 'success', 'info', etc. for colors
        style={'description_width': 'initial'}
    )
);



HTML(value='<h1>🚂 Ridership Map Explorer</h1>')

interactive(children=(ToggleButtons(description='Select a ridership metric to view:', options=('Boards', 'Alig…